# Imports

In [ ]:
import numpy as np
import pandas as pd
import sys, getopt
import os
from glob import glob
from keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.utils import array_to_img
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import load_img
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import *
from keras.layers import *
import keras.backend as K
from keras import optimizers
from keras.activations import *
import tensorflow as tf
import gc
import itertools
import cv2
import math
import matplotlib.pyplot as plt
from sys import getsizeof
%matplotlib inline

In [ ]:
gc.collect()

***
# Reading in Data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
load_weights = f"/kaggle/input/tracknetpickleball/model906_30"
save_weights = "/kaggle/working/dense_weights"
dataDir = f"/kaggle/input/tracknetpickleball/y_data_30"

In [ ]:
r = os.path.abspath(os.path.join(dataDir))
path = glob(os.path.join(r, '*.npy'))
num = len(path) / 2
idx = np.arange(num, dtype='int') + 1

***
# Model Training Hyperparameters

In [ ]:
BATCH_SIZE=10
HEIGHT=288
WIDTH=512
epochs = 20
tol = 15

# keep very small for unfreezing
lr = 0.01

***
# Utility Functions

In [ ]:
#Loss function
def custom_loss(y_true, y_pred):
	loss = (-1)*(K.square(1 - y_pred) * y_true * K.log(K.clip(y_pred, K.epsilon(), 1)) + K.square(y_pred) * (1 - y_true) * K.log(K.clip(1 - y_pred, K.epsilon(), 1)))
	return K.mean(loss)

In [ ]:
def TrackNet3(input_height, input_width): #input_height = 288, input_width = 512
	imgs_input = Input(shape=(9,input_height,input_width))
	#Layer1
	x = Conv2D(64, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(imgs_input)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer2
	x = Conv2D(64, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(x)
	x = ( Activation('relu'))(x)
	x1 = ( BatchNormalization())(x)
	#Layer3
	x = MaxPooling2D((2, 2), strides=(2, 2), data_format='channels_first' )(x1)
	#Layer4
	x = Conv2D(128, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer5
	x = Conv2D(128, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(x)
	x = ( Activation('relu'))(x)
	x2 = ( BatchNormalization())(x)
	#x2 = (Dropout(0.5))(x2) 
	#Layer6
	x = MaxPooling2D((2, 2), strides=(2, 2), data_format='channels_first' )(x2)
	#Layer7
	x = Conv2D(256, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer8
	x = Conv2D(256, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer9
	x = Conv2D(256, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(x)
	x = ( Activation('relu'))(x)
	x3 = ( BatchNormalization())(x)
	#x3 = (Dropout(0.5))(x3)
	#Layer10
	x = MaxPooling2D((2, 2), strides=(2, 2), data_format='channels_first' )(x3)
	#Layer11
	x = ( Conv2D(512, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first'))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer12
	x = ( Conv2D(512, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first'))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer13
	x = ( Conv2D(512, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first'))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#x = (Dropout(0.5))(x)
	#Layer14
	#x = UpSampling2D( (2,2), data_format='channels_first')(x)
	x = concatenate( [UpSampling2D( (2,2), data_format='channels_first')(x), x3], axis=1)
	#Layer15
	x = ( Conv2D( 256, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first'))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer16
	x = ( Conv2D( 256, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first'))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer17
	x = ( Conv2D( 256, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first'))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	
	#Layer18
	#x = UpSampling2D( (2,2), data_format='channels_first')(x)
	x = concatenate( [UpSampling2D( (2,2), data_format='channels_first')(x), x2], axis=1)
	#Layer19
	x = ( Conv2D( 128 , (3, 3), kernel_initializer='random_uniform', padding='same' , data_format='channels_first' ))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer20
	x = ( Conv2D( 128 , (3, 3), kernel_initializer='random_uniform', padding='same' , data_format='channels_first' ))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer21
	#x = UpSampling2D( (2,2), data_format='channels_first')(x)
	x = concatenate( [UpSampling2D( (2,2), data_format='channels_first')(x), x1], axis=1)
	#Layer22
	x = ( Conv2D( 64 , (3, 3), kernel_initializer='random_uniform', padding='same'  , data_format='channels_first' ))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer23
	x = ( Conv2D( 64 , (3, 3), kernel_initializer='random_uniform', padding='same'  , data_format='channels_first' ))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer24
	x =  Conv2D( 3 , (1, 1) , kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(x)
	x = ( Activation('sigmoid'))(x)
        
	o_shape = Model(imgs_input , x ).output_shape
	#print ("layer24 output shape:", o_shape[1],o_shape[2],o_shape[3])
	#Layer24 output shape: (3, 288, 512)
	OutputHeight = o_shape[2]
	OutputWidth = o_shape[3]
	output = x
	model = Model( imgs_input , output)
	#model input unit:9*288*512, output unit:3*288*512
	model.outputWidth = OutputWidth
	model.outputHeight = OutputHeight
	#Show model's details
	#model.summary()
	return model

In [ ]:
#Return the numbers of true positive, true negative, false positive and false negative
def outcome(y_pred, y_true, tol):
	n = y_pred.shape[0]
	i = 0
	TP = TN = FP1 = FP2 = FN = 0
	while i < n:
		for j in range(3):
			if np.amax(y_pred[i][j]) == 0 and np.amax(y_true[i][j]) == 0:
				TN += 1
			elif np.amax(y_pred[i][j]) > 0 and np.amax(y_true[i][j]) == 0:
				FP2 += 1
			elif np.amax(y_pred[i][j]) == 0 and np.amax(y_true[i][j]) > 0:
				FN += 1
			elif np.amax(y_pred[i][j]) > 0 and np.amax(y_true[i][j]) > 0:
				h_pred = y_pred[i][j] * 255
				h_true = y_true[i][j] * 255
				h_pred = h_pred.astype('uint8')
				h_true = h_true.astype('uint8')
				#h_pred
				(cnts, _) = cv2.findContours(h_pred.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
				rects = [cv2.boundingRect(ctr) for ctr in cnts]
				max_area_idx = 0
				max_area = rects[max_area_idx][2] * rects[max_area_idx][3]
				for j in range(len(rects)):
					area = rects[j][2] * rects[j][3]
					if area > max_area:
						max_area_idx = j
						max_area = area
				target = rects[max_area_idx]
				(cx_pred, cy_pred) = (int(target[0] + target[2] / 2), int(target[1] + target[3] / 2))
				#h_true
				(cnts, _) = cv2.findContours(h_true.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
				rects = [cv2.boundingRect(ctr) for ctr in cnts]
				max_area_idx = 0
				max_area = rects[max_area_idx][2] * rects[max_area_idx][3]
				for j in range(len(rects)):
					area = rects[j][2] * rects[j][3]
					if area > max_area:
						max_area_idx = j
						max_area = area
				target = rects[max_area_idx]
				(cx_true, cy_true) = (int(target[0] + target[2] / 2), int(target[1] + target[3] / 2))
				dist = math.sqrt(pow(cx_pred-cx_true, 2)+pow(cy_pred-cy_true, 2))
				if dist > tol:
					FP1 += 1
				else:
					TP += 1
		i += 1
	return (TP, TN, FP1, FP2, FN)

In [ ]:
#Return the values of accuracy, precision and recall
def evaluation(outcome):
    (TP, TN, FP1, FP2, FN) = outcome
    # does this implictly unpack?
    try:
        accuracy = (TP + TN) / (TP + TN + FP1 + FP2 + FN)
    except:
        accuracy = 0
    try:
        precision = TP / (TP + FP1 + FP2)
    except:
        precision = 0
    try:
        recall = TP / (TP + FN)
    except:
        recall = 0
    return (accuracy, precision, recall)

***
# Load Base TrackNetV2

In [ ]:
# load old model
# all params are currently unfrozen
base_model = load_model(load_weights, custom_objects={'custom_loss':custom_loss})

ADADELTA = optimizers.Adadelta(learning_rate=lr) # 1e-5 if fine tuning
base_model.compile(loss=custom_loss,optimizer=ADADELTA)

In [ ]:
len(base_model.layers)

In [ ]:
trainable_params = np.sum([K.count_params(w) for w in base_model.trainable_weights])
trainable_params

In [ ]:
non_trainable_params = np.sum([K.count_params(w) for w in base_model.non_trainable_weights])
non_trainable_params

***
# Train/Val split (70/30)

In [ ]:
npy_batch_size = 250

In [ ]:
val_idx_start = math.floor(.70 * npy_batch_size)
val_idx_start

In [ ]:
train = np.arange(0,val_idx_start)
val = np.arange(val_idx_start, npy_batch_size)

***
# Base Model Performance

In [ ]:
base_metrics = pd.DataFrame(index = ["train", "val", "all"], columns=["TP","TN", "FP1", "FP2", "FN", "loss", "acc", "prec", "rec"], dtype="float64")
base_metrics = base_metrics.fillna(0)
base_metrics   

In [ ]:
# print("EVALUATING BASE MODEL PERFORMANCE")
# for file_num in idx:
#     print(f"\tReading in file: x_data_{str(file_num)}.npy")
#     X = np.load(os.path.abspath(os.path.join(dataDir, 'x_data_' + str(file_num) + '.npy')))

#     print("\tMaking predictions")
#     y_hat = base_model.predict(X, batch_size=BATCH_SIZE)

#     print(f"\tDeleting x_data_{str(file_num)}.npy")
#     del X
#     gc.collect()

#     print(f"\tReading in file: y_data_{str(file_num)}.npy")
#     y = np.load(os.path.abspath(os.path.join(dataDir, 'y_data_' + str(file_num) + '.npy')))

#     print("\tCalculating train/val/overall loss")
#     base_metrics.loc["train", "loss"] += custom_loss(y[train], y_hat[train]).numpy()
#     base_metrics.loc["val", "loss"] += custom_loss(y[val], y_hat[val]).numpy()
#     base_metrics.loc["all", "loss"] += custom_loss(y, y_hat).numpy()

#     print("\tCalculating train/val/overall classification performance")
#     y_pred = (y_hat > 0.5).astype("float32")
#     base_metrics.loc["train",["TP", "TN", "FP1", "FP2", "FN"]] += outcome(y_pred[train], y[train], tol)
#     base_metrics.loc["val", ["TP", "TN", "FP1", "FP2", "FN"]] += outcome(y_pred[val], y[val], tol)
#     base_metrics.loc["all", ["TP", "TN", "FP1", "FP2", "FN"]] += outcome(y_pred, y, tol)

#     print(f"\tDeleting y_data_{str(file_num)}.npy")
#     del y
#     del y_hat
#     del y_pred
#     gc.collect()

# print("\tCalculating aggregate metrics")
# for subset in ["train", "val", "all"]:
#     # acc, prec, rec
#     base_metrics.loc[subset, ["acc", "prec", "rec"]] = evaluation(base_metrics.loc[subset,["TP", "TN", "FP1", "FP2", "FN"]])

#     # standardize TP, FP, ...
#     base_metrics.loc[subset, ["TP", "TN", "FP1", "FP2", "FN"]] /= base_metrics.loc[subset, ["TP", "TN", "FP1", "FP2", "FN"]].sum()

#     # make into percentages
#     base_metrics.loc[subset, ["TP", "TN", "FP1", "FP2", "FN", "acc", "prec", "rec"]] = (base_metrics.loc[subset, ["TP", "TN", "FP1", "FP2", "FN", "acc", "prec", "rec"]]*100).round(2)

# print("BASE METRICS")
# print(base_metrics)  
# print()

In [ ]:
# BASE METRICS (files 1-30, 250 examples each)
#           TP     TN   FP1   FP2     FN      loss    acc   prec    rec
# train  51.68  10.37  5.33  2.32  30.30  0.009474  62.05  87.11  63.04
# val    50.47  13.79  4.46  2.55  28.73  0.009288  64.27  87.81  63.73
# all    51.32  11.40  5.07  2.39  29.83  0.009418  62.72  87.31  63.24

***
# Option 2: Freeze everything except last k conv layers

In [ ]:
k = 14

In [ ]:
conv_indices = [idx for idx, layer in enumerate(base_model.layers) if "conv" in layer.name]
conv_indices

In [ ]:
# number of conv layers that can be fine tuned
len(conv_indices)

In [ ]:
freeze_to = conv_indices[-k]
freeze_to

In [ ]:
for layer in base_model.layers[0:freeze_to]:
    layer.trainable = False
for layer in base_model.layers[freeze_to:]:
    layer.trainable= True

In [ ]:
for l in base_model.layers:
    print(f"{l.trainable}: {l.name}")

In [ ]:
# compile the new model
model = base_model
ADADELTA = optimizers.Adadelta(learning_rate=lr) # 1e-5 if fine tuning
model.compile(loss=custom_loss, optimizer=ADADELTA)

In [ ]:
trainable_params = np.sum([K.count_params(w) for w in model.trainable_weights])
trainable_params

In [ ]:
non_trainable_params = np.sum([K.count_params(w) for w in model.non_trainable_weights])
non_trainable_params

***
# Fine-Tune the Model

In [ ]:
train_metrics = pd.DataFrame(columns = ["TP","TN", "FP1", "FP2", "FN", "loss", "acc", "prec", "rec"], index = np.arange(1, epochs+1))
train_metrics = train_metrics.fillna(0)
train_metrics

In [ ]:
val_metrics = pd.DataFrame(columns = ["TP","TN", "FP1", "FP2", "FN", "loss", "acc", "prec", "rec"], index = np.arange(1, epochs+1))
val_metrics = val_metrics.fillna(0)
val_metrics

In [ ]:
print('Beginning training......')
for epoch in range(1, epochs+1):
    print('============ epoch', epoch, '================')
    
    print("TRAINING")
    for file_num in idx:
        print(f"\tReading in files: x_data_{str(file_num)}.npy, y_data_{str(file_num)}.npy")
        # read in big batch (500 examples)
        X = np.load(os.path.abspath(os.path.join(dataDir, 'x_data_' + str(file_num) + '.npy')))
        y = np.load(os.path.abspath(os.path.join(dataDir, 'y_data_' + str(file_num) + '.npy')))
        
        print("\tFitting model with 70% train")
        # fit in smaller batches (3 examples)
        history = model.fit(X[train], 
                            y[train], 
                            batch_size=BATCH_SIZE, 
                            epochs=1,
                            shuffle=False)
        
        print(f"\tDeleting x_data_{str(file_num)}.npy and y_data_{str(file_num)}.npy")
        del X
        del y
        gc.collect()
        
    print("EVALUATING")
    for file_num in idx:
        print(f"\tReading in file: x_data_{str(file_num)}.npy")
        X = np.load(os.path.abspath(os.path.join(dataDir, 'x_data_' + str(file_num) + '.npy')))

        print("\tMaking predictions")
        y_hat = model.predict(X, batch_size=BATCH_SIZE)
        
        print(f"\tDeleting x_data_{str(file_num)}.npy")
        del X
        gc.collect()
        
        print(f"\tReading in file: y_data_{str(file_num)}.npy")
        y = np.load(os.path.abspath(os.path.join(dataDir, 'y_data_' + str(file_num) + '.npy')))

        print("\tCalculating train and valid loss")
        train_metrics.loc[epoch, "loss"] += custom_loss(y[train], y_hat[train]).numpy()
        val_metrics.loc[epoch, "loss"] += custom_loss(y[val], y_hat[val]).numpy()
        
        print("\tCalculating train and valid classification performance")
        y_pred = (y_hat > 0.5).astype("float32")
        train_metrics.loc[epoch, ["TP", "TN", "FP1", "FP2", "FN"]] += outcome(y_pred[train], y[train], tol)
        val_metrics.loc[epoch, ["TP", "TN", "FP1", "FP2", "FN"]] += outcome(y_pred[val], y[val], tol)

        print(f"\tDeleting y_data_{str(file_num)}.npy")
        del y
        del y_hat
        del y_pred
        gc.collect()
    
    print("METRICS")
    # acc, prec, rec
    train_metrics.loc[epoch, ["acc", "prec", "rec"]] = evaluation(train_metrics.loc[epoch,["TP", "TN", "FP1", "FP2", "FN"]])
    val_metrics.loc[epoch, ["acc", "prec", "rec"]] = evaluation(val_metrics.loc[epoch,["TP", "TN", "FP1", "FP2", "FN"]])

    # standardize TP, FP, ...
    train_metrics.loc[epoch, ["TP", "TN", "FP1", "FP2", "FN"]] /= train_metrics.loc[epoch, ["TP", "TN", "FP1", "FP2", "FN"]].sum()
    val_metrics.loc[epoch, ["TP", "TN", "FP1", "FP2", "FN"]] /= val_metrics.loc[epoch, ["TP", "TN", "FP1", "FP2", "FN"]].sum()

    # make into percentages
    train_metrics.loc[epoch, ["TP", "TN", "FP1", "FP2", "FN", "acc", "prec", "rec"]] = (train_metrics.loc[epoch, ["TP", "TN", "FP1", "FP2", "FN", "acc", "prec", "rec"]]*100).round(2)
    val_metrics.loc[epoch, ["TP", "TN", "FP1", "FP2", "FN", "acc", "prec", "rec"]] = (val_metrics.loc[epoch, ["TP", "TN", "FP1", "FP2", "FN", "acc", "prec", "rec"]]*100).round(2)
    
    epoch_metrics = pd.concat([train_metrics.loc[epoch,:], val_metrics.loc[epoch,:]], axis=1)
    epoch_metrics.columns = ["train", "val"]
    print(epoch_metrics.astype('object')) 
    print()

    # Save intermediate weights during training
    model.save(save_weights + '_' + str(epoch))

***
# Classification Performance Metrics

In [ ]:
train_metrics

In [ ]:
val_metrics

In [ ]:
# BASE METRICS (files 1-30, 250 examples each)
#           TP     TN   FP1   FP2     FN      loss    acc   prec    rec
# train  51.68  10.37  5.33  2.32  30.30  0.009474  62.05  87.11  63.04
# val    50.47  13.79  4.46  2.55  28.73  0.009288  64.27  87.81  63.73
# all    51.32  11.40  5.07  2.39  29.83  0.009418  62.72  87.31  63.24

In [ ]:
clf_perf = train_metrics.join(val_metrics, lsuffix="_train", rsuffix="_val")         
clf_perf

In [ ]:
clf_perf.to_csv("/kaggle/working/clf_perf.csv")

In [ ]:
def compare_train_val(metric, train_metrics, val_metrics):
    plt.title(metric)
    plt.xlabel('epoch')
    plt.ylabel("%")
    x = np.arange(1,train_metrics.shape[0]+1,1)
    plt.plot(x, train_metrics[metric], label=f"train_{metric}")
    plt.plot(x, val_metrics[metric], label=f"val_{metric}")
    plt.legend()
    plt.show()

In [ ]:
for metric in train_metrics.columns:
    compare_train_val(metric, train_metrics, val_metrics)

In [ ]:
model.save(save_weights)